In [68]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist
from ISLP import load_data

import siuba
from siuba import _, if_else, select, mutate
from ISLP.models import (ModelSpec as MS,summarize)

In [69]:
smarket = load_data('Smarket')

In [71]:
smarket2=(
    smarket >>
    #mutate(Direction = if_else(_.Direction == 'Up',1,0)) >>
    mutate(train = if_else(_.Year < 2005,1,0) ) >>
    select(~_.Year,~_.Today)

)
# print(smarket2.head())

# print(design)
train = (smarket2.train == 1)
design = MS(smarket2 >> select(~_.Direction,~_.train))
X = design.fit_transform(smarket2)
y = smarket2['Direction']  == 'Up'


X_train, X_test = X.loc[train], X.loc[~train]
y_train, y_test = y.loc[train], y.loc[~train]
X_train = np.expand_dims(X_train, axis=0)
y_train = np.expand_dims(y_train, axis=0)

In [72]:
input_size =1
num_labels = len(np.unique(y_train))

In [73]:
batch_size = 64
hidden_units = 128
dropout = 0.45


In [74]:
model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(num_labels))
# this is the output for one-hot vector
model.add(Activation('softmax'))
model.summary()
#plot_model(model, to_file='mlp-mnist.png', show_shapes=True)
# loss function for one-hot vector
# use of adam optimizer
# accuracy is good metric for classification tasks
model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])
# train the network


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 128)               256       
                                                                 
 activation_38 (Activation)  (None, 128)               0         
                                                                 
 dropout_26 (Dropout)        (None, 128)               0         
                                                                 
 dense_39 (Dense)            (None, 128)               16512     
                                                                 
 activation_39 (Activation)  (None, 128)               0         
                                                                 
 dropout_27 (Dropout)        (None, 128)               0         
                                                                 
 dense_40 (Dense)            (None, 2)               

In [75]:
model.fit(X_train, y_train, epochs=20, batch_size=batch_size)

Epoch 1/20


ValueError: in user code:

    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 277, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception